In [1]:
cd "/suphys/aloe8475"


/suphys/aloe8475


In [2]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [3]:
from scipy.io import loadmat, savemat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
import warnings
from IPython.core.debugger import set_trace
import nct
import bct

#warnings.filterwarnings('ignore')

In [4]:
import pickle 
import _pickle as cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [5]:
# this_seed2=700
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [6]:
#Select Largest Components
def select_largest_component_new(wires_dict):
    """
    Find and select largest connected component of the original graph G.
    Throws away unconnected components and updates all the keys in wires_dict 
    """
#     def connected_component_subgraphs(G):
#         for c in nx.connected_components(G):
#             yield G.subgraph(c)
    
    wires_dict['G'] = max(connected_component_subgraphs(wires_dict['G']), key=len)
#     set_trace()
    nw = len(wires_dict['G'].nodes())
    nj = len(wires_dict['G'].edges())   
    
    logging.info("The largest component has %5d nodes and %6d edges", nw, nj)

    # Replace values in the dictionary
    wires_dict['number_of_wires']     = nw
    wires_dict['number_of_junctions'] = nj
    wires_dict['xa'] = wires_dict['xa'][wires_dict['G'].nodes()] 
    wires_dict['ya'] = wires_dict['ya'][wires_dict['G'].nodes()] 
    wires_dict['xb'] = wires_dict['xb'][wires_dict['G'].nodes()] 
    wires_dict['yb'] = wires_dict['yb'][wires_dict['G'].nodes()]
    wires_dict['xc'] = wires_dict['xc'][wires_dict['G'].nodes()] 
    wires_dict['yc'] = wires_dict['yc'][wires_dict['G'].nodes()]
 
    # Keep old edge_list
    old_edge_list = [(ii, kk) for ii, kk in  zip(wires_dict['edge_list'][:, 0], wires_dict['edge_list'][:, 1])]
    # Remove old edge list
    wires_dict = wires.remove_key(wires_dict, 'edge_list') 
    # Save indices of intersections in the old graph
    ind_dict = {key:value for value,key in enumerate(old_edge_list)}
    new_edge_list = sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0])
    # Find intersection between the two sets
    inter = set(ind_dict).intersection(new_edge_list)
    # Retrieve edge indices/positions from the old list
    edges_idx = [ind_dict[idx] for idx in inter]
       
    # These have length equal to number of junctions -- only get the ones we need
    wires_dict['xi'] = wires_dict['xi'][edges_idx] 
    wires_dict['yi'] = wires_dict['yi'][edges_idx] 
    
    # Get contiguous numbering of nodes
    # Build node mapping 
    node_mapping    = {key:value for value, key in enumerate(sorted(wires_dict['G'].nodes()))}
    # This  step also renames edges list
    wires_dict['G'] =  nx.relabel_nodes(wires_dict['G'] , node_mapping)

    # Swap node vertices if vertex 0 is larger than vertex 1, then sort by first element
    wires_dict['edge_list'] = np.asarray(sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0]))
    
    # Save adjacency matrix of new graph
    wires_dict = wires.remove_key(wires_dict, 'adj_matrix') 
    wires_dict = wires.generate_adj_matrix(wires_dict)

    wire_distances = wires.cdist(np.array([wires_dict['xc'], wires_dict['yc']]).T, np.array([wires_dict['xc'], wires_dict['yc']]).T, metric='euclidean')    
    wires_dict['wire_distances'] = wire_distances

    return wires_dict 

In [7]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [8]:
data=loadmat('avgDegreeVals_allASN.mat')
avgDeg=data['avgdeg'][0]

numNodes=data['numNodes'][0]

In [9]:
BetaVals=np.linspace(0.1,0.9,9)

WS_BetaSweep=[[None]*len(avgDeg) for i in range(len(BetaVals))] #11 different betas



In [10]:
# seed=[]
# for i in range(len(avgDeg)):
#     seed.append(np.random.randint(10000))

In [11]:
#get seed from previous networks:
name='WS_BetaSweepNetworks.pkl'
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[WS_BetaNetworks] = pickle.load(file)
seed=[]
for i in range(len(avgDeg)):
        seed.append(WS_BetaNetworks[0][i]['seed'])

In [12]:
#create watts strogatz graphs:

#Test with 100nw network

BetaGraphs=[[None]*len(avgDeg) for i in range(len(BetaVals))]
for i in tqdm(range(len(BetaVals))):
    temp=[]
    for j in range(len(avgDeg)):
        k=int(np.round(avgDeg[j]))
        temp.append(nx.watts_strogatz_graph(numNodes[j],2*k,i,seed=seed[j]))
        BetaGraphs[i][j]=(max(connected_component_subgraphs(temp[j]), key=len))

In [13]:
onAmp=[[None]*len(avgDeg) for i in range(len(BetaVals))]
for i in tqdm(range(len(BetaVals))):
    temp=[]
    shortestPath=[]
    for j in tqdm(range(len(avgDeg))):
        temp.append(getFarthestPairing(nx.adj_matrix(BetaGraphs[i][j]).todense()))
        shortestPath.append(nx.shortest_path_length(BetaGraphs[i][j],temp[j][0],temp[j][1]))
        onAmp[i][j]=(shortestPath[j]/5)

In [14]:
WS_BetaNetworks=[[None]*len(avgDeg) for i in range(len(BetaVals))]
for i in range(len(BetaVals)):
    for j in range(len(avgDeg)):
        WS_BetaNetworks[i][j]={'seed':[],'adj_matrix':[],'G':[],'Accuracy':{'Linear Transformation':[],'Memory Capacity':[]},'Graph Theory':{'Small World':[],'Modularity':[],'CCoeff':[],'MZ':[],'PCoeff':[],'PL':[]}}

In [15]:
# WS:
for i in tqdm(range(len(BetaVals))):
    for j in range(len(avgDeg)):
        WS_BetaNetworks[i][j]['seed']=seed[j]
        WS_BetaNetworks[i][j]['sp_amp']=onAmp[i][j]
name='WS_BetaSweepNetworks.pkl'
print('Saving WS')
with open(name, 'wb') as f:
    pickle.dump([WS_BetaNetworks], f)  
print('Saved')


Saving WS
Saved


In [16]:
def WS_Run(network,onAmp,nodesList,WS_BetaNetworks,i,j):
    stimulus=[[],[]]
    dt = 1e-2
    f=0.5
    Time=5
    stimulus[0]=(stimulus__(biasType='AC',onAmp=onAmp[i][j],TimeVector=np.arange(0,Time,dt),f=f))
    stimulus[1]=(stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt))) #we don't want this drain to be active during training

    stimulus2 = [item for item in stimulus] #go through each list in the list and find the ith item

    results=(runSim(connectivity__(graph=network[i][j]),stimulus=stimulus2, contactMode='farthest', T = Time, dt = dt, onAmp = onAmp[i][j], biasType='AC',f=f,junctionMode='tunneling'))
#     period=1/f
    nwSqu=[None]*len(nodesList[i][j])
    countk=0
    for k in nodesList[i][j]: #loop through sets of nodes for regression
        ResultSqu=[]
        ResultSqu = nonLinearTrans(results,'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu[countk]=(ResultSqu['accuracy'])
        countk=countk+1
        if k == nodesList[1][i][-1]: #if k is equal to the largest node sampling
            WS_BetaNetworks[i][j]['Accuracy']['Linear Transformation']=[]
            WS_BetaNetworks[i][j]['Accuracy']['Linear Transformation'].append(nwSqu)
#             print(WS_Random[i]['Accuracy']['Linear Transformation'])
        else:
            WS_BetaNetworks[i][j]['Accuracy']['Linear Transformation']=[None]
   
    acc=nwSqu
    return acc

In [17]:
nodesList=[[None]*len(avgDeg) for i in range(len(BetaVals))]
for i in range(len(BetaVals)): #for each beta value
    for j in range(len(avgDeg)): 
                if len(BetaGraphs[i][j]) >= 250:
                    nodesList[i][j]=[50,100,150,200,250,len(BetaGraphs[i][j])]#range(50, len(ws300[i][0])+1,50)
                elif len(BetaGraphs[i][j]) >= 200 and len(BetaGraphs[i][j]) < 250:
                    nodesList[i][j]=[50,100,150,200,len(BetaGraphs[i][j])]#range(50, len(ws300[i][0])+1,50)  
                elif len(BetaGraphs[i][j]) >= 150 and len(BetaGraphs[i][j]) < 200:
                    nodesList[i][j]=[50,100,150,len(BetaGraphs[i][j])]#range(50, len(ws300[i][0])+1,50)  
                elif len(BetaGraphs[i][j]) >= 100 and len(BetaGraphs[i][j]) < 150:
                    nodesList[i][j]=[50,100,len(BetaGraphs[i][j])]#range(50, len(ws300[i][0])+1,5

In [19]:
results=[]
# r1=[];r2=[];r3=[];r4=[];r5=[];r6=[];r7=[];r8=[];r9=[];
for i in tqdm(range(len(BetaVals))):
    init=[]
    for j in range(len(avgDeg)):
        init.append(inputPacker(WS_Run,BetaGraphs,onAmp,nodesList,WS_BetaNetworks,i,j))
    results.append(para_run(WS_Run,init,nCPU=16))
#     if i  == 0:
#         r1=para_run(WS_Run,init,nCPU=20)
#     elif i == 1:
#         r2=para_run(WS_Run,init,nCPU=20)
#     elif i == 2:
#         r3=para_run(WS_Run,init,nCPU=20)
#     elif i == 3:
#         r4=para_run(WS_Run,init,nCPU=20)
#     elif i == 4:
#         r5=para_run(WS_Run,init,nCPU=20)
#     elif i == 5:
#         r6=para_run(WS_Run,init,nCPU=20)
#     elif i == 6:
#         r7=para_run(WS_Run,init,nCPU=20)
#     elif i == 7:
#         r8=para_run(WS_Run,init,nCPU=20)
#     elif i == 8:
#         r9=para_run(WS_Run,init,nCPU=20)

2020-06-10 15:03:46,569:INFO:First current path [0, 290, 281, 272, 263, 254, 245, 236, 227, 218, 209, 200, 191, 182, 173, 164, 155, 146] formed at time = 0.75 s.
2020-06-10 15:03:50,026:INFO:First current path [0, 7, 21, 35, 49, 63, 77, 91, 105, 118, 131, 142] formed at time = 0.76 s.
2020-06-10 15:03:56,792:INFO:First current path [0, 281, 261, 241, 221, 201, 181, 161, 142] formed at time = 0.75 s.
2020-06-10 15:04:04,411:INFO:First current path [0, 10, 33, 56, 78, 100, 122, 140] formed at time = 0.76 s.
2020-06-10 15:04:14,484:INFO:First current path [0, 16, 48, 80, 112, 130] formed at time = 0.77 s.
2020-06-10 15:04:19,943:INFO:First current path [0, 7, 44, 80, 115, 150] formed at time = 0.78 s.
2020-06-10 15:04:24,280:INFO:First current path [0, 261, 217, 173, 134] formed at time = 0.76 s.
2020-06-10 15:04:30,589:INFO:First current path [0, 2, 54, 106] formed at time = 0.78 s.
2020-06-10 15:04:43,434:INFO:First current path [0, 235, 170, 132] formed at time = 0.77 s.
2020-06-10 15:

2020-06-10 15:05:30,957:INFO:First current path [0, 266, 232, 197, 162, 142] formed at time = 0.77 s.
2020-06-10 15:05:31,333:INFO:First current path [0, 3, 41, 78, 116] formed at time = 0.79 s.


2020-06-10 15:05:42,784:INFO:First current path [0, 254, 208, 161, 143] formed at time = 0.78 s.


2020-06-10 15:05:49,744:INFO:First current path [0, 2, 52, 102] formed at time = 0.78 s.
2020-06-10 15:05:50,425:INFO:First current path [1, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, 89, 94, 99, 104, 109, 114, 119, 124, 129, 134, 139, 144, 147] formed at time = 0.76 s.


2020-06-10 15:05:56,035:INFO:First current path [0, 5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93, 101, 109, 117, 125, 133, 141, 146] formed at time = 0.76 s.


...

2020-06-10 20:20:26,900:INFO:First current path [2, 111, 54, 130] formed at time = 0.76 s.


2020-06-10 20:20:32,282:INFO:First current path [5, 182, 286, 33] formed at time = 0.75 s.


2020-06-10 20:20:41,031:INFO:First current path [1, 224, 284, 73] formed at time = 0.75 s.


2020-06-10 20:20:47,977:INFO:First current path [9, 245, 148, 48] formed at time = 0.75 s.


2020-06-10 20:20:53,351:INFO:First current path [4, 27, 181, 44] formed at time = 0.77 s.


In [27]:
len(results[0][0])

6

In [28]:
new_result=np.asarray(results).reshape(9,300,6)

In [49]:
new_result[0][299]

array([0.67755094, 0.73292249, 0.77202057, 0.78846542, 0.79678505,
       0.80016791])

In [50]:
for i in range(len(BetaVals)):
    for j in range(len(new_result[0])):
        WS_BetaNetworks[i][j]['Accuracy']['Linear Transformation']=[ii for ii in new_result[i][j]]

In [51]:
WS_BetaNetworks[0][299]['Accuracy']['Linear Transformation']

[0.6775509389487463,
 0.7329224904390386,
 0.7720205667932645,
 0.7884654232183556,
 0.7967850523836921,
 0.8001679064607784]

In [53]:
#Save networks so we don't have to run this every time   
name='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/WS_BetaSweep_networks_LinearTransformation.pkl'
if (not os.path.isfile(name)): #if we haven't saved the file
    print('Saving Max Min Random + Grid Accuracies')
    with open(name, 'wb') as f:
        pickle.dump([WS_BetaNetworks], f)   
else:
    print('Loading Random + Grid Max Min Accuracies')
    file = open(name, 'rb')
    [WS_BetaNetworks] = pickle.load(file)
    print('Loaded')

Saving Max Min Random + Grid Accuracies


In [52]:
#export adj matrices to calculate small worldness in matlab:
adj_mats={"AdjMat_WS":[[None]*len(avgDeg) for i in range(len(BetaVals))]}
for i in range(len(BetaVals)):
    for j in tqdm(range(len(avgDeg))):
        adj_mats['AdjMat_WS'][i][j]=(dict(graphs=nx.adj_matrix(BetaGraphs[i][j]).todense()))
        WS_BetaNetworks[i][j]['adj_matrix']=nx.adj_matrix(BetaGraphs[i][j]).todense()
savemat('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/BetaSweepNetworks.mat',adj_mats)

In [ ]:
cd "/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/"

In [ ]:
# temp=loadmat(r'300_WS_smallworld.mat')
temp=loadmat(r'BetaSweepNetworks_smallworld.mat')
smallworld_sweep=temp['smallworld_sweep']
del temp

In [54]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [ ]:
# WS:
for i in tqdm(range(len(WS_BetaNetworks))):
    ci = []
    pcoeff= []
    mz= []
    clustering = []
    q=[]
    for j in tqdm(range(len(WS_BetaNetworks[i]))):
        ci,q=nct.community_louvain(np.squeeze(np.asarray(WS_BetaNetworks[i][j]['adj_matrix'])))
        pcoeff=bct.participation_coef(np.squeeze(np.asarray(WS_BetaNetworks[i][j]['adj_matrix'])),ci)
        mz=bct.module_degree_zscore(np.squeeze(np.asarray(WS_BetaNetworks[i][j]['adj_matrix'])),ci)
        WS_BetaNetworks[i][j]['G']=BetaGraphs[i][j]
        clustering=nx.clustering(WS_BetaNetworks[i][j]['G'])
        WS_BetaNetworks[i][j]['Graph Theory']['PL']=dict(nx.all_pairs_shortest_path_length(WS_BetaNetworks[i][j]['G']))
        WS_BetaNetworks[i][j]['Graph Theory']['Modularity']=ci
        WS_BetaNetworks[i][j]['Graph Theory']['Modularity Score']=q
        WS_BetaNetworks[i][j]['Graph Theory']['PCoeff']=pcoeff
        WS_BetaNetworks[i][j]['Graph Theory']['MZ']=mz
        WS_BetaNetworks[i][j]['Graph Theory']['Small World']=smallworld_sweep[0][i][j]
        WS_BetaNetworks[i][j]['Graph Theory']['CCoeff']=clustering
        WS_BetaNetworks[i][j]['Graph Theory']['Degree']=nx.degree( WS_BetaNetworks[i][j]['G'])
        WS_BetaNetworks[i][j]['Graph Theory']['AvgPL']=nx.average_shortest_path_length(WS_BetaNetworks[i][j]['G'])